In [1]:
import torch
from torch import nn, optim
from torch.optim import Adam
from torch.utils.data import DataLoader, random_split, ConcatDataset
from torchvision import transforms, datasets
import glob
from torchvision import models
from torchvision.models import efficientnet_v2_m, EfficientNet_V2_M_Weights
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/4995Proj/Project/Data/project_data.zip" "/content/project_data.zip"

In [4]:
!mkdir -p "/content/data"

In [5]:
!unzip -q "/content/project_data.zip" -d "/content/data"

In [6]:
project_data = {}

project_data['spring'] = glob.glob('/content/data/project_data/spring/spring*')
project_data['summer'] = glob.glob('/content/data/project_data/summer/summer*')
project_data['fall'] = glob.glob('/content/data/project_data/fall/fall*')
project_data['winter'] = glob.glob('/content/data/project_data/winter/winter*')

print(f"Spring #:  {len(project_data['spring'])}")
print(f"Summer #:  {len(project_data['summer'])}")
print(f" Fall #:  {len(project_data['fall'])}")
print(f"Winter #:  {len(project_data['winter'])}")

Spring #:  6000
Summer #:  6000
 Fall #:  6000
Winter #:  6000


In [7]:
# Specify data directory
main_dir = '/content/data/project_data'

In [8]:
# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
# Define the categories (seasons)
categories = ['spring', 'summer', 'fall', 'winter']

# Specify the ratio for splitting
train_ratio = 0.7
val_ratio = 0.15

# Define your transformations
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Empty lists to hold data from each category
train_data = []
val_data = []
test_data = []

In [10]:
# Loop over each category
for i, category in enumerate(categories):
    # Define the full dataset
    full_dataset = datasets.ImageFolder(main_dir, transform=transform)

    # Get the lengths of splits
    train_len = int(train_ratio * len(full_dataset))
    val_len = int(val_ratio * len(full_dataset))
    test_len = len(full_dataset) - train_len - val_len

    # Perform the split
    train_dataset, val_dataset, test_dataset = random_split(full_dataset, lengths=[train_len, val_len, test_len])

    # Add category-specific datasets to the main datasets
    train_data.append(train_dataset)
    val_data.append(val_dataset)
    test_data.append(test_dataset)

In [11]:
# Concatenate all the subsets to create final train, val, and test datasets
train_data = ConcatDataset(train_data)
val_data = ConcatDataset(val_data)
test_data = ConcatDataset(test_data)

# Use DataLoader to convert these datasets into batches, shuffle them, and load in parallel
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(val_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

In [12]:
# Load the pre-trained model
model = efficientnet_v2_m(weights=EfficientNet_V2_M_Weights.DEFAULT)

# Replace the last layer to match the number of classes in your dataset
num_classes = len(categories)
model.classifier = nn.Linear(model.classifier[1].in_features, num_classes)

# Move the model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# Define the number of training epochs
num_epochs = 6

Downloading: "https://download.pytorch.org/models/efficientnet_v2_m-dc08266a.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_m-dc08266a.pth
100%|██████████| 208M/208M [00:01<00:00, 120MB/s]


In [13]:
def calculate_accuracy(outputs, labels):
    _, preds = torch.max(outputs, 1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [14]:
# Initialize best validation loss and patience counter
best_val_loss = float('inf')
patience_counter = 0
patience_limit = 2

# Train the model
for epoch in range(num_epochs):
    train_loss = 0.0
    train_acc = 0.0
    val_loss = 0.0
    val_acc = 0.0

    # Training phase
    model.train()
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # compute loss and accuracy
        train_loss += loss.item() * inputs.size(0)
        train_acc += calculate_accuracy(outputs, labels)

    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader)

    # Validation phase
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # compute loss and accuracy
            val_loss += loss.item() * inputs.size(0)
            val_acc += calculate_accuracy(outputs, labels)

    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_acc / len(val_loader)

    # Save model if it has the best validation loss so far
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        patience_counter = 0  # Reset patience counter
    else:
        patience_counter += 1

    print('Epoch: {}/{}'.format(epoch+1, num_epochs))
    print('Training Loss: {:.4f} - Training Accuracy: {:.4f}'.format(train_loss, train_acc))
    print('Validation Loss: {:.4f} - Validation Accuracy: {:.4f}'.format(val_loss, val_acc))

    # Check for early stopping
    if patience_counter >= patience_limit:
        print('Early stopping triggered')
        break

print('Finished Training')

Epoch: 1/6
Training Loss: 0.6048 - Training Accuracy: 0.7673
Validation Loss: 0.4143 - Validation Accuracy: 0.8440
Epoch: 2/6
Training Loss: 0.3522 - Training Accuracy: 0.8683
Validation Loss: 0.2442 - Validation Accuracy: 0.9123
Epoch: 3/6
Training Loss: 0.2084 - Training Accuracy: 0.9261
Validation Loss: 0.1558 - Validation Accuracy: 0.9476
Epoch: 4/6
Training Loss: 0.1467 - Training Accuracy: 0.9498
Validation Loss: 0.1215 - Validation Accuracy: 0.9588
Epoch: 5/6
Training Loss: 0.1097 - Training Accuracy: 0.9625
Validation Loss: 0.1090 - Validation Accuracy: 0.9645
Epoch: 6/6
Training Loss: 0.0910 - Training Accuracy: 0.9669
Validation Loss: 0.0987 - Validation Accuracy: 0.9655
Finished Training


In [15]:
# Load the best model
model.load_state_dict(torch.load('best_model.pth'))

# Testing phase
model.eval()
test_loss = 0.0
test_acc = 0.0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # compute loss and accuracy
        test_loss += loss.item() * inputs.size(0)
        test_acc += calculate_accuracy(outputs, labels)

test_loss = test_loss / len(test_loader.dataset)
test_acc = test_acc / len(test_loader)

print('Testing Loss: {:.4f} - Testing Accuracy: {:.4f}'.format(test_loss, test_acc))

Testing Loss: 0.1029 - Testing Accuracy: 0.9613
